# COVID India Model
This notebook enables you to run different scenarios of an SEIR model of the spread of COVID-19 in India.

In order to run the model lets first load all required code

In [ ]:
# The Scenario parameters are set here before invoking the simulation
"""
main module to automatically run the model scenarios

"""
__version__ = "6.0.00.2020"
__doc__ = "Automatically generates different scenario outputs for COVID-19 India Model"
__name__ = "COVID-19 India Scenario Generator"

# contains the main model
import covid_india_model as covid
# contains the functions that initialise the model variables
import covid_india_model_config as cmc
# contains all functions to change parameter values for scenarios
import covid_india_scenarios as cs
# contains the functions to simulate the model and produce the output
import simulator as sim
# builtin functions for system dynamics
import SD as sd

Next lets set sim.show or sim.save to True depending on whether we want to view or save the output to file. You can do both, but depending on the computing power, the program may crash. So choose to do one at a time. 

In [ ]:
sim.show = True
sim.save = False

Lets define some housekeeping functions to obtain the outputs in the desired format. 

In [ ]:
def merge_scenarios(df):
    """
    merge_scenarios [removes the contact numbers from scenario names to make comparison of all scenarios with contact numbers possible]

    [extended_summary]

    Args:
        df ([DataFrame]): [DataFrame with the scenario data]

    Returns:
        [type]: [description]
    """    
    import pandas as pd
    element_list = df.index
    for contact_rate in range(111, -10, -10):
        element_list = [word.replace("-"+str(contact_rate), '')
                        for word in element_list]
    df.index = element_list
    df.index.name = 'scenario'
    df.reset_index(inplace=True)
    return df

def summary_table(df1, df2, df3, df4):
    """
    summary_table [saves the DataFrames for new cases, total cases and total deaths into excel sheets]

    [extended_summary]

   Args:
        df1 ([DataFrame): [DataFrame with peak and peeaktime values for new cases]
        df2 ([DataFrame]): [DataFrame with peak and peeaktime values for total cases]
        df3 ([DataFrame]): [DataFrame with peak and peeaktime values for total deaths]
     """    
    import pandas as pd
    sim.save_to_excel(df1, 'infection_rate', 'Scenario Summary', 'w')
    sim.save_to_excel(df2, 'total_infected', 'Scenario Summary', 'w')
    sim.save_to_excel(df3, 'covid_deaths', 'Scenario Summary', 'w')
    sim.save_to_excel(df4, 'infected', 'Scenario Summary', 'w')


def summary_pivot(df1, df2, df3, df4):
    """
    summary_pivot [geerates excel files with peak and peak time comparison for all scenarios]

    [extended_summary]

    Args:
        df1 ([DataFrame): [DataFrame with peak and peeaktime values for new cases]
        df2 ([DataFrame]): [DataFrame with peak and peeaktime values for total cases]
        df3 ([DataFrame]): [DataFrame with peak and peeaktime values for total deaths]
    """    
    import pandas as pd
    new_cases = pd.pivot_table(df1, values=['peak', 'peak_time'], index=[
                               'scenario'], columns=['contact_rate 2'])
    total_cases = pd.pivot_table(df2, values=['peak', 'peak_time'], index=[
                                 'scenario'], columns=['contact_rate 2'])
    total_deaths = pd.pivot_table(df3, values=['peak', 'peak_time'], index=[
                                  'scenario'], columns=['contact_rate 2'])
    infected = pd.pivot_table(df4, values=['peak', 'peak_time'], index=[
                                  'scenario'], columns=['contact_rate 2'])

    sim.save_to_excel(new_cases, 'infection_rate', 'Scenario Pivot', 'w')
    sim.save_to_excel(total_cases, 'total_infected', 'Scenario Pivot', 'w')
    sim.save_to_excel(total_deaths, 'covid_deaths', 'Scenario Pivot', 'w')
    sim.save_to_excel(infected, 'infected', 'Scenario Pivot', 'w')


Now let's automate the simulator to initialise, run, and store each scenario for different lockdown stringencies.  

In [ ]:

def initialise_scenario(program, scene, value, intervention_time):
    """
    initialise_scenario [initialise the scenario settings and invoke the simulator]

    [extended_summary]

    Args:
        program ([string]): [name of the scenario]
        scene ([function]): [scenario parameter setting function to be invoked]
        value ([float]): [parameter values to be set]
        intervention_time ([int]): [the intervention time to be passed to the scenario parmeter setting function]
    """    
    cmc.duration(1, 1825, 0.5)
    for l2 in range(1, 110, 10):
        program_name = program + " - " + str(value) + "-" + str(l2)
        cmc.initialise_model(program_name)  
        scene(value, intervention_time)
        # This is what you use to set change the death multiplier
        # ty = 1, 1, 1, 1  # there is no multiplier effect
        # program_name = "C-" + program_name
        ty = 1, 5, 8, 10  # comment the line above and uncomment this to set a death multiplier effect due to undercapacity
        program_name = "U-" + program_name
        cs.deaths_by_undercapacity(ty)
        # This is the 1st lockdown and required to match the historic data, must always be on
        cs.lockdown1(10, 85)    # lockdown was initiated from day 85
        cs.closure1(0.99,85)    # closure of international flights was also initiated from day 85
        # This is the 4th lockdown from day 140 when there was relaxation of contact rate: must always be on 
        cs.lockdown2(l2, 140)   
        scenario = program_name  # "contacts/day:" + str(l2)
        # program + "-" + str(value)   #+ "-" + "under weak lockdown"
        filename = program_name
        print('initiating scenario ', filename, " with ", l2, " contacts/day")
        sim.simulate(covid.model, __name__, scenario,
                     cmc.no_of_time_points, cmc.dt)
        # store the scenario as a dataframe in a dictionary
        sc = sim.store_scenario(scenario, cmc.store(), cmc.description, cmc.dt)

Let's invoke and run each scenario now.

In [ ]:
sim.reset(covid.__version__)
intervention_time = 140

initialise_scenario('L', cs.lockdown1, 1, intervention_time)  # for lockdown to be eased
initialise_scenario('C', cs.closure2, 0.99, intervention_time)  # for closure to contine
initialise_scenario('C', cs.closure2, 0, intervention_time)  # for closure to be lifted
initialise_scenario('I', cs.immunity, 0.1, intervention_time)  # for 10% effectiveness immunity programs to be introduced
initialise_scenario('I', cs.immunity, 0.5, intervention_time)  # for 50% effectiveness immunity programs to be introduced
initialise_scenario('Q', cs.quarantine, 0.1, intervention_time)  # for 10% effectiveness isolation programs to be introduced
initialise_scenario('Q', cs.quarantine, 0.5, intervention_time)  # for 50% effectiveness isolation programs to be introduced
initialise_scenario('M', cs.masks, 0.1, intervention_time)  # for 10% effectiveness nasks programs to be introduced
initialise_scenario('M', cs.masks, 0.5, intervention_time)  # for 50% effectiveness masks programs to be introduced

# Ensure the data can be summarized in plots and tables
df1 = sim.summarize('infection_rate', 'contact_rate 2', cmc.dt)
df2 = sim.summarize('total_infected', 'contact_rate 2', cmc.dt)
df3 = sim.summarize('covid_deaths', 'contact_rate 2', cmc.dt)
df4 = sim.summarize('infected', 'contact_rate 2', cmc.dt)

merge_scenarios(df1)
merge_scenarios(df2)
merge_scenarios(df3)
merge_scenarios(df4)

## Peak new cases per day and time to peak new cases per day
Compare the peak value, and the time to peak, to be expected for new cases per day in different scenarios 

In [ ]:
sim.plot_x_log_y(df1, 'contact_rate 2', 'peak', 'peak values under different scenarios',
                     'contacts per day', 'new cases per day', 'new cases scenario comparison - log', False)
sim.plot_x_log_y(df1, 'contact_rate 2', 'peak_time', 'peak time under different scenarios',
                     'contacts per day', 'time to peak new cases per day (days)', 'new cases scenario comparison - log', False)

## Peak total cases  and time to peak total cases 
Compare the peak value, and the time to peak, to be expected for total cases in different scenarios. This is a good indicator of the duration of the pandemic. 

In [ ]:
sim.plot_x_log_y(df2, 'contact_rate 2', 'peak', 'peak values under different scenarios',
                    'contacts per day', 'total cases', 'total cases scenario comparison - log', False)
sim.plot_x_log_y(df2, 'contact_rate 2', 'peak_time', 'peak time under different scenarios',
                    'contacts per day', 'time to peak total cases (days)', 'total cases scenario comparison - log', False)

## Peak total deaths and time to peak total deaths
Compare the peak value, and the time to peak, to be expected for total deaths in different scenarios. This is a good indicator of the last deaths due to COVID-19. 

In [ ]:
sim.plot_x_log_y(df3, 'contact_rate 2', 'peak', 'peak values under different scenarios',
                    'contacts per day', 'total deaths', 'total deaths scenario comparison - log', False)
sim.plot_x_log_y(df3, 'contact_rate 2', 'peak_time', 'peak time under different scenarios',
                    'contacts per day', 'time to peak total deaths (days)', 'total deaths scenario comparison - log', False)

## Peak active cases and time to peak active cases 
Compare the peak value, and the time to peak, to be expected for active cases in different scenarios. This is a good indicator of the burden of the pandemic and actual flattening accomplished. 

In [ ]:
sim.plot_x_log_y(df4, 'contact_rate 2', 'peak', 'peak values under different scenarios',
                    'contacts per day', 'active cases', 'active cases scenario comparison - log', False)
sim.plot_x_log_y(df4, 'contact_rate 2', 'peak_time', 'peak time under different scenarios',
                    'contacts per day', 'time to peak active cases (days)', 'active cases scenario comparison - log', False)
# uncomment the next line if you need a graph with legends
# sim.plot_x_log_y(df1, 'contact_rate 2', 'peak', 'peak values under different scenarios',
                    'contacts per day', 'new cases per day', 'new cases scenario comparison - legend', True)

## Peak values and peak time tables
Generate and store peak time and peak value tables as excel files

In [ ]:
summary_table(df1, df2, df3, df4)
summary_pivot(df1, df2, df3, df4)

## Plotting the output
**Note**: The model is storing data for 198 scenarios for 36,510 time points for 35 variables. Unless you have very good computing power with memory, please plot only graphs you want to see,comment the rest. Trying to plot all graphs may cause the plotting module or python to crash.

## Daily new confirmed COVID-19 cases
Compare the new cases per day in different scenarios in linear and log plots. 

In [ ]:
sim.ploty('infection_rate', '', 'Days', 'Daily new cases', 'All scenarios')
# Log plot causes python to crash, do not plot other plots if you want to do log plots
# sim.plot_log_y('infection_rate', '', 'Days',
#                'Daily new cases', 'All scenarios' + '-log plot')

## Cumulative confirmed COVID-19 cases
Compare the total cases in different scenarios in linear and log plots

In [ ]:
sim.ploty('total_infected', '', 'Days', 'Total cases', 'All scenarios')
# Log plot causes python to crash, do not plot other plots if you want to do log plots
# sim.plot_log_y('total_infected', '', 'Days',
#                 'Total cases', 'All scenarios' + '-log plot')

## Cumulative confirmed COVID-19 deaths
Compare the total deaths in different scenarios in linear and log plots

In [ ]:
sim.ploty('covid_deaths', '', 'Days', 'Total deaths', 'All scenarios')
# Log plot causes python to crash, do not plot other plots if you want to do log plots
# sim.plot_log_y('covid_deaths', '', 'Days',
#                 'Total deaths', 'All scenarios' + '-log plot')
# sim.plotx_logy('scenario', 'covid_deaths', '',
#                 'scenario', 'covid_deaths', 'All scenarios')

## Active COVID-19 cases
Compare the active cases in different scenarios in linear and log plots

In [ ]:
sim.ploty('infected', '', 'Days', 'Active cases', 'All scenarios')
# Log plot causes python to crash, do not plot other plots if you want to do log plots
# sim.plot_log_y('infected', '', 'Days',
#                 'Acive cases', 'All scenarios' + '-log plot')
# sim.plotx_logy('scenario', 'infected', '',
#                 'scenario', 'Active cases', 'All scenarios')

## Susceptible population
Compare susceptible population with different scenarios 

In [ ]:
sim.ploty('susceptible', '', 'Days', 'Susceptible population', 'All Scenarios')

## Exposed population
Compare exposed population with different scenarios 

In [ ]:
sim.ploty('exposed', '', 'Days', 'Exposed population', 'All Scenarios')

## Infected population
Compare infected population with different scenarios 

In [ ]:
sim.ploty('infected', '', 'Days', 'Active cases', 'All Scenarios')

## Recovered population
Compare recovered population with different scenarios 

In [ ]:
sim.ploty(' recovered', '', 'Days', 'Recovered population', 'All Scenarios')

## Scenario tables
Store the scenario values for all scenarios in excel files

In [ ]:
if sim.save == True:
    sim.compare_scenarios_to_excel("All scenarios" + ".xlsx")

print(">> I'm done simulating all scenarios")
